# 1) Overlap Between Maps - ANCs

For a new redistricting map, show what old ANCs overlap with each new ANC.

Make sure rtree is installed in your local environment. See this notebook for your previous struggles: https://github.com/devinbrady/dc-parking/blob/main/Spatial-Join.ipynb

In [1]:
import pandas as pd
import geopandas as gpd
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
# Use this projection for measuring distances in meters
# https://octo.dc.gov/page/coordinate-system-standards
dc_crs = ('epsg', '26985')

# Use WGS84 for saving GeoJSON for display in QGIS
output_crs = ('epsg', '4326')

In [3]:
# Read in data
previous_map = gpd.read_file('anc-2012.geojson')
new_map = gpd.read_file('anc-2022.geojson')

In [5]:
# # Set column names on the new map
# new_map['smd_name'] = new_map['SMD_ID']
# new_map['smd_id'] = 'smd_2022_' + new_map['SMD_ID']
# new_map.rename(columns={'ANC_ID': 'anc_id'}, inplace=True)

# # We only need these columns going forward
# map_columns = ['smd_id', 'smd_name', 'anc_id', 'geometry']

# # Save out the new map for later steps
# # new_map[map_columns].to_file('to-mapbox-2022-smd-data-overlap.geojson', driver='GeoJSON')

In [6]:
# Convert to local coordinate reference system
previous_map = previous_map.to_crs(dc_crs)
new_map = new_map.to_crs(dc_crs)

In [7]:
previous_map['district_area'] = previous_map.geometry.area
new_map['district_area'] = new_map.geometry.area

In [8]:
def calculate_overlap(map_a, map_b, suffix_a, suffix_b, id_field):

    overlap = map_a[overlap_columns].overlay(
        map_b[overlap_columns], how='identity', keep_geom_type=False
    )
    
    overlap.rename(columns={
        id_field + '_1': id_field + suffix_a
        , id_field + '_2': id_field + suffix_b
        , 'district_area_1': 'district_area' + suffix_a
        , 'district_area_2': 'district_area' + suffix_b
    }, inplace=True)

    overlap['overlap_area'] = overlap.geometry.area
    overlap['overlap_perc'] = overlap['overlap_area'] / overlap['district_area' + suffix_a]
    
    overlap['district_rank'] = overlap.groupby(id_field + suffix_a).overlap_perc.rank(ascending=False)
    overlap.sort_values(by=[id_field + suffix_a, 'overlap_area'], ascending=[True, False], inplace=True)
    
    overlap.drop('geometry', axis=1, inplace=True)
    
    return overlap

In [9]:
# Calculate the overlap
overlap_columns = ['anc_id', 'district_area', 'geometry']

In [10]:
# Backwards - how much of each new district is made up of the old districts? 
# Use this for the frontend list for each new district. 
overlap_backwards = calculate_overlap(
    new_map[overlap_columns], previous_map[overlap_columns], '_2022', '_2012', 'anc_id')

In [11]:
# Forwards - how much of each old district went into this new district? 
# Use this for colors for continuity
overlap_forwards = calculate_overlap(
    previous_map[overlap_columns], new_map[overlap_columns], '_2012', '_2022', 'anc_id')

In [12]:
# Save results
columns_for_csv = [c for c in overlap_backwards.columns if c != 'geometry']
overlap_backwards[columns_for_csv].to_csv('overlap_backwards_anc.csv', index=False)
overlap_forwards[columns_for_csv].to_csv('overlap_forwards_anc.csv', index=False)

In [13]:
# Should be 46
(overlap_backwards.district_rank == 1).sum()

46

In [14]:
# Should be 40
(overlap_forwards.district_rank == 1).sum()

40

## Examples

Every old district that is in this new district:

In [15]:
overlap_forwards[overlap_forwards.anc_id_2022 == 'anc_1C_2022']

,anc_id_2012,district_area_2012,anc_id_2022,district_area_2022,overlap_area,overlap_perc,district_rank
16,anc_1A,1663391.93,anc_1C_2022,1224285.30,0.06,0.00,5.00
17,anc_1B,2671190.98,anc_1C_2022,1224285.30,0.08,0.00,6.00
11,anc_1C,1285108.96,anc_1C_2022,1224285.30,1222983.56,0.95,1.00
12,anc_1D,947589.98,anc_1C_2022,1224285.30,0.06,0.00,5.00
13,anc_2B,2160613.45,anc_1C_2022,1224285.30,0.08,0.00,6.00
14,anc_2D,801770.73,anc_1C_2022,1224285.30,0.36,0.00,5.00
15,anc_3C,5293720.75,anc_1C_2022,1224285.30,1301.09,0.00,4.00


Every new district that is made up of parts of this old district:

In [16]:
overlap_forwards[overlap_forwards.anc_id_2012 == 'anc_1C']

,anc_id_2012,district_area_2012,anc_id_2022,district_area_2022,overlap_area,overlap_perc,district_rank
11,anc_1C,1285108.96,anc_1C_2022,1224285.30,1222983.56,0.95,1.00
5,anc_1C,1285108.96,anc_1B_2022,1464265.71,60488.04,0.05,2.00
35,anc_1C,1285108.96,anc_3C_2022,4725916.27,744.92,0.00,3.00
0,anc_1C,1285108.96,anc_1A_2022,1125853.81,678.23,0.00,4.00
18,anc_1C,1285108.96,anc_1D_2022,1075325.75,213.94,0.00,5.00
30,anc_1C,1285108.96,anc_2D_2022,795976.64,0.25,0.00,6.00
24,anc_1C,1285108.96,anc_2B_2022,1318654.34,0.02,0.00,7.00
238,anc_1C,1285108.96,NaN,nan,0.00,0.00,8.00


Every new district that this old district went to make: 

In [24]:
overlap_backwards[overlap_backwards.anc_id_2022 == 'anc_1C_2022']

,anc_id_2022,district_area_2022,anc_id_2012,district_area_2012,overlap_area,overlap_perc,district_rank
9,anc_1D_2022,1075325.75,anc_1D,947589.98,947550.24,0.88,1.00
22,anc_1D_2022,1075325.75,anc_1A,1663391.93,127437.69,0.12,2.00
3,anc_1D_2022,1075325.75,anc_1C,1285108.96,213.94,0.00,3.00
70,anc_1D_2022,1075325.75,anc_3C,5293720.75,123.82,0.00,4.00
77,anc_1D_2022,1075325.75,anc_4A,7739216.58,0.03,0.00,5.00
14,anc_1D_2022,1075325.75,anc_4C,3139126.04,0.03,0.00,6.00
241,anc_1D_2022,1075325.75,NaN,nan,0.00,0.00,7.00


In [21]:
overlap_backwards[overlap_backwards.anc_id_2012 == 'anc_1C']

,anc_id_2022,district_area_2022,anc_id_2012,district_area_2012,overlap_area,overlap_perc,district_rank
0,anc_1A_2022,1125853.81,anc_1C,1285108.96,678.23,0.00,3.00
1,anc_1B_2022,1464265.71,anc_1C,1285108.96,60488.04,0.04,2.00
2,anc_1C_2022,1224285.30,anc_1C,1285108.96,1222983.56,1.00,1.00
3,anc_1D_2022,1075325.75,anc_1C,1285108.96,213.94,0.00,3.00
4,anc_2B_2022,1318654.34,anc_1C,1285108.96,0.02,0.00,5.00
5,anc_2D_2022,795976.64,anc_1C,1285108.96,0.25,0.00,3.00
6,anc_3C_2022,4725916.27,anc_1C,1285108.96,744.92,0.00,4.00


In [23]:
overlap_forwards.overlap_perc.sum()

40.000000886563384

In [22]:
overlap_backwards.overlap_perc.sum()

46.00000000017819